In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [10]:
load_dotenv(override=True)

True

In [3]:
# Define 3 instructions (to be used for agents)
instructions_formal = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions_cool = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions_concise = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."



In [5]:
# Define 3 agents from the above instructions
agent_formal = Agent(name= "Formal_Agent", instructions=instructions_formal, model="gpt-4o-mini")
agent_cool = Agent(name= "Cool_Agent", instructions=instructions_cool, model="gpt-4o-mini")
agent_concise = Agent(name= "Concise_Agent", instructions=instructions_concise, model="gpt-4o-mini")


In [8]:
# Define 3 tools from the above agents
tool_description_formal = "Write a professional, serious cold sales email."
tool_formal = agent_formal.as_tool(tool_name="Formal_Tool", tool_description=tool_description_formal)

tool_description_cool = "Write a witty, engaging cold sales email."
tool_cool = agent_cool.as_tool(tool_name="Cool_Tool", tool_description=tool_description_cool)

tool_description_concise = "Write a concise, to the point cold sales email."
tool_concise = agent_concise.as_tool(tool_name="Concise_Tool", tool_description=tool_description_concise)



#### Handoff
Handoff is an agent with an additional "handoff_description" congif.

This handoff will do three things (using three tools):
- generates a subject
- converts the email to html format
- sends the email using SendGrid

In [16]:
# Define a tool to generate a subject

subject_writer_instruction = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response. "
subject_writer_agent = Agent(name="subject_writer_agent",
                             instructions= subject_writer_instruction,
                             model= "gpt-4o-mini")
subject_writer_tool_description = "Write a subject for a cold sales email"
subject_writer_tool = subject_writer_agent.as_tool(tool_name="subject_writer_tool", tool_description=subject_writer_tool_description)


In [17]:
# Define a tool to convert the email to html format
html_converter_instruction = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."
html_convertor_agent = Agent(name="html_convertor_agent",)
html_convertor_tool_description = "Convert a text email body to an HTML email body"
html_convertor_tool = html_convertor_agent.as_tool(tool_name="html_convertor_tool", tool_description=html_convertor_tool_description)

In [66]:
# Define a send_email_tool tool using the decorator

@function_tool
def send_email_tool(subject: str, html_body: str) -> Dict[str, str]:
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("motaslimi@leibnizrule.com")
    to_email = To(email="motaslimi@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [67]:
handoff_tools = [subject_writer_tool, html_convertor_tool, send_email_tool]

In [68]:
handoff_instruction = "You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer_tool tool to write a subject for the email, then use the html_convertor_tool tool to convert the body to HTML. \
Finally, you use the send_email_tool tool to send the email with the subject and HTML body."

In [69]:
handoff_description = "Convert an email to HTML and send it"

In [70]:
handoff_send_email = Agent(name="send_email_handoff",
                           instructions= handoff_instruction ,
                           model = "gpt-4o-mini",
                           tools= handoff_tools,
                           handoff_description= handoff_description
                           )

####Final Agent

The final agent uses three tools and one handoff
- tool_formal
- tool_cool
- tool_concise
- handoff_send_email

In [71]:
final_agent_instruction = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at before choosing the best one. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the handoff_send_email to format and send the email."

In [72]:
final_agent_tools = [tool_formal, tool_cool, tool_concise]

In [73]:
final_agent = Agent(name="Cold Email Sales Manager",
                         instructions=final_agent_instruction,
                         tools= final_agent_tools,
                         model="gpt-4o-mini",
                         handoffs=[handoff_send_email])

In [74]:
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Cold Sales Email Sender Agent"):
    result = await Runner.run(final_agent, message)
